## Employee Onboarding: From Tangled Yarn to Clear Roadmap with LangGraph!

### The Old Way:
Onboarding is often manual, slow, and confusing – like a tangled ball of yarn.

### Our New Way:
We use a smart system powered by LangGraph to make it smooth and automated – like a clear roadmap.

---

### Demo Flow: Alex's Onboarding Journey

1.  **Demo Start: Offer Accepted!**
    *   Alex accepts her job offer! This kicks off the automated process.

2.  **Step 1: Start Alex's Journey**
    *   The system gets Alex's basic info.

3.  **Step 2: Decide the Path (Decision Point!)**
    *   The system quickly checks if Alex's role needs a background check.

4.  **Step 3: (If Needed) Background Check**
    *   If **YES**, the system starts the background check.
    *   If **NO**, it skips this and moves on.

5.  **Step 4: (If Needed) Check Result (Potential Failure End)**
    *   If a background check was done, the system sees if it passed.
    *   If it **FAILED**, the process stops here.

6.  **Step 5: Do Two Things at Once! (Parallel Action!)**
    *   If the background check passed (or wasn't needed), the system now does two important things *at the same time*:
        *   Tell HR to start the paperwork.
        *   Tell IT to start setting up Alex's computer access.

7.  **Step 6: Bring It Together & Finalize (Parallel Join & Final Action)**
    *   The system waits until *both* HR and IT confirm they've started their tasks.
    *   Once confirmed, it does the final steps, like scheduling Alex's first-day welcome.

8.  **Step 7: Onboarding Complete! (Success End)**
    *   Alex is ready for her first day! The process finishes successfully.

---


![image](images\employee-onboarding.png)

In [2]:
# graph TD
#     A["1.Start Onboarding (START)"] --> B["2.Decide Role Path (Needs BG Check?)"];

#     B -- "YES" --> C["3.Run Background Check"];
#     C --> D["4.Check BG Result (Did it Pass?)"];
#     D -- "Passed" --> E["5.Trigger Parallel Setup (Kick off HR & IT)"];
#     D -- "Failed" --> FAIL_END["(END) Onboarding Failed"];

#     %% If NO BG check, skip to parallel setup
#     B -- "NO" --> E;

#     %% Parallel Execution Block
#     E --> F["6.Initiate HR Paperwork"];
#     E --> G["7.Initiate IT Access"];

#     %% Join after parallel tasks
#     F --> H["8.Finalize Onboarding (Schedule First Day)"];
#     G --> H;

#     H --> SUCCESS_END["(END) Onboarding Complete"];


In [ ]:
import random
import time
from typing import TypedDict, Literal

from langgraph.graph import StateGraph, END



In [ ]:
# --- 1. Define the State ---
# This is like the "file" or "notepad" that travels with Alex's onboarding process,
# holding all the important information as it moves from step to step.

class EmployeeOnboardingState(TypedDict):
    employee_name: str
    role_requires_bg_check: bool  # Input: Does this type of role need a BG check?
    
    # Internal tracking flags
    decision_bg_check_needed: bool | None
    bg_check_status: Literal["NOT_RUN", "PENDING", "PASSED", "FAILED"]
    hr_task_initiated: bool
    it_task_initiated: bool
    onboarding_outcome: Literal["PENDING", "SUCCESS", "FAILED_BG_CHECK"]



In [ ]:
# --- 2. Define the Nodes (The Steps or "Stations") ---
# Each function below is a "station" in our onboarding roadmap.
# It takes the current state (Alex's file) and can update it.

def start_onboarding_node(state: EmployeeOnboardingState) -> EmployeeOnboardingState:
    """Step 1: Start Alex's Journey"""
    employee_name = state["employee_name"]
    print(f"\n--- 🚀 Starting Onboarding for: {employee_name} ---")
    # Initialize some default states
    return {
        "decision_bg_check_needed": None,
        "bg_check_status": "NOT_RUN",
        "hr_task_initiated": False,
        "it_task_initiated": False,
        "onboarding_outcome": "PENDING",
    }

def decide_bg_check_path_node(state: EmployeeOnboardingState) -> EmployeeOnboardingState:
    """Step 2: Decide the Path (Needs BG Check?)"""
    print("--- 🤔 Step 2: Deciding if Background Check is needed... ---")
    if state["role_requires_bg_check"]:
        print("   -> Role requires a background check.")
        return {"decision_bg_check_needed": True}
    else:
        print("   -> Role does NOT require a background check. Skipping.")
        return {"decision_bg_check_needed": False}

def run_background_check_node(state: EmployeeOnboardingState) -> EmployeeOnboardingState:
    """Step 3: (If Needed) Background Check"""
    print("--- ⏳ Step 3: Running Background Check... ---")
    print("   (Simulating external check, this might take a moment...)")
    time.sleep(1) # Simulate work
    # For demo, we'll just mark it as pending, next step checks result
    return {"bg_check_status": "PENDING"}

def check_bg_result_node(state: EmployeeOnboardingState) -> EmployeeOnboardingState:
    """Step 4: (If Needed) Check Result"""
    print("--- ตรวจสอบ Step 4: Checking Background Check Result... ---")
    # Simulate a result
    passed = random.choice([True, True, False]) # 2/3 chance of passing
    if passed:
        print("   -> Background Check: PASSED 👍")
        return {"bg_check_status": "PASSED"}
    else:
        print("   -> Background Check: FAILED 👎")
        return {"bg_check_status": "FAILED", "onboarding_outcome": "FAILED_BG_CHECK"}

def trigger_parallel_hr_it_node(state: EmployeeOnboardingState) -> EmployeeOnboardingState:
    """Step 5: Do Two Things at Once! (This node acts as the fork)"""
    print("---  paralelas Step 5: Kicking off HR & IT tasks in parallel... ---")
    # No state change needed here, just a point to branch from.
    return {}

def initiate_hr_paperwork_node(state: EmployeeOnboardingState) -> EmployeeOnboardingState:
    """Parallel Task 1: Tell HR to start paperwork"""
    print("   ---  параллельно HR: Initiating HR Paperwork... ---")
    time.sleep(0.5) # Simulate HR work
    print("   --- параллельно HR: Paperwork process started by HR. ---")
    return {"hr_task_initiated": True}

def initiate_it_access_node(state: EmployeeOnboardingState) -> EmployeeOnboardingState:
    """Parallel Task 2: Tell IT to set up access"""
    print("   --- параллельно IT: Initiating IT Access Setup... ---")
    time.sleep(0.7) # Simulate IT work
    print("   --- параллельно IT: Access setup started by IT. ---")
    return {"it_task_initiated": True}

def finalize_onboarding_node(state: EmployeeOnboardingState) -> EmployeeOnboardingState:
    """Step 6: Bring It Together & Finalize"""
    # This node implicitly waits for both HR and IT tasks because of how we'll wire the graph.
    print("--- ✅ Step 6: HR & IT tasks initiated. Finalizing Onboarding... ---")
    print("   (Scheduling first-day welcome, sending final details, etc.)")
    time.sleep(0.5)
    return {"onboarding_outcome": "SUCCESS"}



In [ ]:
# --- 3. Define Conditional Logic Functions ---
# These functions help LangGraph decide which path to take at a "Decision Point".

def should_run_bg_check(state: EmployeeOnboardingState) -> Literal["run_bg_check", "skip_bg_check"]:
    """Checks the decision from decide_bg_check_path_node"""
    if state["decision_bg_check_needed"] is True:
        return "run_bg_check"
    return "skip_bg_check"

def bg_check_outcome(state: EmployeeOnboardingState) -> Literal["passed", "failed"]:
    """Checks the result from check_bg_result_node"""
    if state["bg_check_status"] == "PASSED":
        return "passed"
    return "failed" # Covers "FAILED" or any other unexpected status



In [ ]:
# --- 4. Build the Graph (The Roadmap) ---
# Now we tell LangGraph how the "stations" are connected.

workflow = StateGraph(EmployeeOnboardingState)

# Add the nodes (stations)
workflow.add_node("start_onboarding", start_onboarding_node)
workflow.add_node("decide_bg_path", decide_bg_check_path_node)
workflow.add_node("run_bg_check", run_background_check_node)
workflow.add_node("check_bg_result", check_bg_result_node)
workflow.add_node("trigger_parallel_hr_it", trigger_parallel_hr_it_node) # The fork
workflow.add_node("hr_paperwork", initiate_hr_paperwork_node)
workflow.add_node("it_access", initiate_it_access_node)
workflow.add_node("finalize_onboarding", finalize_onboarding_node) # The join & final step

# Set the starting point of the roadmap
workflow.set_entry_point("start_onboarding")

# Connect the stations with arrows (edges)
workflow.add_edge("start_onboarding", "decide_bg_path")

# Conditional edge for background check
workflow.add_conditional_edges(
    "decide_bg_path",
    should_run_bg_check, # Function that decides
    {
        "run_bg_check": "run_bg_check",       # If "run_bg_check", go to this station
        "skip_bg_check": "trigger_parallel_hr_it" # If "skip_bg_check", go to this station
    }
)

workflow.add_edge("run_bg_check", "check_bg_result")

# Conditional edge after background check result
workflow.add_conditional_edges(
    "check_bg_result",
    bg_check_outcome, # Function that decides
    {
        "passed": "trigger_parallel_hr_it", # If "passed", go to parallel setup
        "failed": END  # If "failed", onboarding stops here (Failure End)
    }
)

# Parallel execution:
# From trigger_parallel_hr_it, go to BOTH hr_paperwork AND it_access
workflow.add_edge("trigger_parallel_hr_it", "hr_paperwork")
workflow.add_edge("trigger_parallel_hr_it", "it_access")

# Join after parallel tasks:
# Both hr_paperwork AND it_access must lead to finalize_onboarding.
# LangGraph handles the "wait" implicitly: finalize_onboarding won't run
# until all its direct predecessors (hr_paperwork, it_access) are done.
workflow.add_edge("hr_paperwork", "finalize_onboarding")
workflow.add_edge("it_access", "finalize_onboarding")

# Final step leads to the end of the roadmap
workflow.add_edge("finalize_onboarding", END) # (Success End)

# --- 5. Compile and Run the Roadmap! ---
app = workflow.compile()

# Let's try it for Alex, whose role requires a background check
print("\n\n--- Scenario 1: Alex (Role needs BG Check) ---")
initial_state_alex = {
    "employee_name": "Alex",
    "role_requires_bg_check": True
}
# The .stream() method lets us see the state after each step
for event in app.stream(initial_state_alex):
    # event is a dictionary, and the key is the node name
    node_name = list(event.keys())[0]
    state_after_node = event[node_name]
    print(f"--- After Node: {node_name} ---")
    print(f"Current State: {state_after_node}\n")
print("--- Alex's Onboarding Process Complete ---")


# Let's try it for Bob, whose role does NOT require a background check
print("\n\n--- Scenario 2: Bob (Role does NOT need BG Check) ---")
initial_state_bob = {
    "employee_name": "Bob",
    "role_requires_bg_check": False
}
for event in app.stream(initial_state_bob):
    node_name = list(event.keys())[0]
    state_after_node = event[node_name]
    print(f"--- After Node: {node_name} ---")
    print(f"Current State: {state_after_node}\n")
print("--- Bob's Onboarding Process Complete ---")

# You can also get the final state directly with .invoke()
# final_state_alex = app.invoke(initial_state_alex)
# print("\nFinal state for Alex:", final_state_alex)
# final_state_bob = app.invoke(initial_state_bob)
# print("Final state for Bob:", final_state_bob)
